In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = np.loadtxt('in_class.txt')
x_train = torch.tensor(data[:,0],dtype=torch.float32) #[nt,]
t_train = torch.tensor(data[:,1],dtype=torch.float32) #[nt,]
u_train = torch.tensor(data[:,2],dtype=torch.float32) #[nt,]
print(x_train.shape)
print(t_train.shape)
print(u_train.shape)

In [ ]:
def residual(model,inputs,params):
    x,t = inputs
    u=model(x,t)
    dudt = torch.autograd.grad(u,t,grad_outputs=torch.ones_like(y),create_graph=True)[0]
    dudx = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(y),create_graph=True)[0]
    d2udx2 = torch.autograd.grad(dudx,x,grad_outputs=torch.ones_like(y),create_graph=True)[0]
    
    return dudt+u*dudx-.01/torch.pi*d2udx2


def boundaries(model,x,t):
    u = model(x,t)
    if x==1 or x==-1:
        return u
    if t==0:
        return -1*torch.sin(torch.pi*x)
    return 0

def create_boundaries(bound_size):
    x = torch.linspace(-1,bound_size,1).view(-1,1)
    zeros = torch.zeros_like(x)
    t1 = torch.ones_like(x)
    t2 = torch.full_like(x,-1)
    pnts1 = torch.cat([zeros,x],dim=1)
    pnts2 = torch.cat([t1,zeros],dim=1)
    pnts3 = torch.cat([t1,zeros],dim=1)
    boundaries = torch.cat([pnts1, pnts2, pnts3], dim=0)
    boundaries.requires_grad_(True)
    return boundaries

def create_collocation(nx,nt):
    x = torch.linspace(-1,1,nx)
    t = torch.linspace(0,1,nt)



In [ ]:
plt.figure()
plt.plot(t_train,y_train[:,0])
plt.plot(t_train,y_train[:,1])
plt.xlabel('time')
plt.ylabel('population')
plt.show()

In [ ]:
class PINN(nn.Module):
    def __init__(self,nin, nout,layers):
        super(PINN,self).__init__()
        m_layers = []
        m_layers.append(nn.Linear(nin,layers[0]))
        # print(f'[{nin},{layers[0]}]')
        m_layers.append(nn.SiLU())
        for i in range(len(layers)-1):
            m_layers.append(nn.Linear(layers[i],layers[i+1]))
            # print(f'[{layers[i]},{layers[i+1]}]')
            m_layers.append(nn.SiLU())
        m_layers.append(nn.Linear(layers[-1],nout))
        # print(f'[{layers[-1]},{nout}]')
        self.network=nn.Sequential(*m_layers)

    def forward(self, y0, tsteps):
        

        return odeint(self.odefunc,y0,tsteps) # [nt,2]
        
    
    def odefunc(self,t,y): #output dy/dt
        return self.network(y)
